# Step 0 - Understanding Req

## Mandatory Tasks

1. Calculate the total revenue generated from subscriptions for each country.
2. Calculate the total number of trials given to users who came from Instagram.
3. Create a new column named 'acquisition_channel' by categorizing users based on their 'attribution_source':
   - 'Paid': users from instagram or tiktok
   - 'Organic': users from organic sources
4. Analyze the trial-to-subscription conversion rate:
   - Calculate the overall conversion rate
   - Break down the conversion rate by attribution_source
5. Calculate the median subscription duration (in months) for each country
6. Calculate the Average Lifetime Value (LTV) by country

## Optional Tasks

7. Predict the churn probability for user #1002 (Clark Kent)
   - Use any relevant features from the dataset
   - Explain your model selection and feature engineering process
   - Provide confidence intervals if applicable

8. Predict the potential Lifetime Value (pLTV) for user #1001 (Bruce Wayne)
   - Explain your methodology and assumptions

## Evaluation Criteria

1. **Code Quality**

2. **Analysis Quality**

3. **Communication**

# Step 1 - Importing libs

In [597]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Set plotting style
plt.style.use('seaborn') 
sns.set_palette('deep')

C:\Users\batuh\AppData\Local\Temp\ipykernel_70920\1839458688.py:9: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


# Step 2 - Connection to DB


In [598]:
conn = sqlite3.connect('papcorns.sqlite')

# Step 3 - Creating DataFrame from db

In [599]:
users_df = pd.read_sql_query("SELECT * FROM users;", conn)

In [600]:
events_df = pd.read_sql_query("SELECT * FROM user_events;", conn)

# Step 4 - Exploring dataFrames

### For users data

In [601]:
users_df.shape

(1002, 5)

In [602]:
list(users_df.columns)

['id', 'created_at', 'attribution_source', 'country', 'name']

In [603]:
users_df.head()

,id,created_at,attribution_source,country,name
0,1,2024-05-07T00:00:00,instagram,US,Eve Brown
1,2,2024-10-12T00:00:00,instagram,NL,Frank Moore
2,3,2024-10-15T00:00:00,tiktok,TR,Ivy Anderson
3,4,2024-08-28T00:00:00,tiktok,TR,Alice Brown
4,5,2024-04-03T00:00:00,organic,NL,Bob Moore


In [604]:
users_df.describe() #No meainingfull numeric value to work

,id
count,1002.000000
mean,501.500000
std,289.396786
min,1.000000
25%,251.250000
50%,501.500000
75%,751.750000
max,1002.000000


In [605]:
users_df.isna().sum() #No null row in columns

id                    0
created_at            0
attribution_source    0
country               0
name                  0
dtype: int64

In [606]:
users_df["attribution_source"].value_counts() # 3 Different value with nearly same amount

tiktok       352
organic      344
instagram    306
Name: attribution_source, dtype: int64

In [607]:
users_df['country'].value_counts() # Again 3 different value with nearly same amount

TR    354
US    340
NL    308
Name: country, dtype: int64

### For event data

In [608]:
events_df.shape # Way more row than user's data

(3486, 5)

In [609]:
list(events_df.columns) 

['id', 'created_at', 'user_id', 'event_name', 'amount_usd']

In [610]:
events_df.head() # 'user_id' propably corrsepond to users_df 'id'

,id,created_at,user_id,event_name,amount_usd
0,1,2024-05-07T00:00:00,1,app_install,NaN
1,2,2024-05-12T00:00:00,1,trial_started,NaN
2,3,2024-05-24T00:00:00,1,trial_cancelled,NaN
3,4,2024-10-12T00:00:00,2,app_install,NaN
4,5,2024-10-13T00:00:00,2,trial_started,NaN


In [611]:
#Lets check if it was true
events_df['user_id'].value_counts()

# As we can see lenght is 1002 and in users_df our shape is (1002, 5) so we have every user with some event in events_df

832     8
331     8
950     8
359     8
819     8
       ..
492     1
489     1
477     1
469     1
1002    1
Name: user_id, Length: 1002, dtype: int64

In [612]:
events_df.describe() 
'''
We have a some numerical value we can make some insights 'amount_usd'.
Maybe we can find some relation to other categorical value 
'''

"\nWe have a some numerical value we can make some insights 'amount_usd'.\nMaybe we can find some relation to other categorical value \n"

In [613]:
events_df.dtypes

#we have two object type one is time-stamp other one is event_name lets check it

id              int64
created_at     object
user_id         int64
event_name     object
amount_usd    float64
dtype: object

In [614]:
events_df['event_name'].value_counts() 
'''
We have 6 different event we can categorize it when we preprocess our data. 
Maybe some correlation can be between amount of usd with event.
'''

'\nWe have 6 different event we can categorize it when we preprocess our data. \nMaybe some correlation can be between amount of usd with event.\n'

In [615]:
events_df.isna().sum()

#We have 2255 row of null in 'amount_usd', it can be 0 for not spending any or can be a null value we will inspect it in a moment.

id               0
created_at       0
user_id          0
event_name       0
amount_usd    2255
dtype: int64

In [616]:
events_df[events_df['amount_usd'].notna()]['event_name'].value_counts()

# So there are two 'event_name' that doesnt make amount null.

subscription_renewed    750
subscription_started    481
Name: event_name, dtype: int64

# Step 5 - Solving Tasks

### Task 1

Calculate the total revenue generated from subscriptions for each country.

In [617]:
money_gain_events = events_df[events_df['event_name'].isin(['subscription_started', 'subscription_renewed'])] 

'''
Öncelikle para kazandıran eventlerin olduğu ayrı bir dataframe oluşturalım.
'''

money_gain_events.head()

,id,created_at,user_id,event_name,amount_usd
5,6,2024-10-20T00:00:00,2,subscription_started,8.99
6,7,2024-11-19T00:00:00,2,subscription_renewed,8.99
7,8,2024-12-19T00:00:00,2,subscription_renewed,8.99
8,9,2025-01-18T00:00:00,2,subscription_renewed,8.99
12,13,2024-10-21T00:00:00,3,subscription_started,4.99


In [618]:
money_gain_data = money_gain_events.merge(users_df[['id', 'country']], left_on='user_id', right_on='id', how='left')

'''
Hangi userın hangi ülkeden olduğunu bulunduğunu bulmak için user_df'teki id ve country kısmını events_df'teki user_id ile birleştirelim.
Böylece hem hangi user olduğunu ve hangi ülkenin kullanıcısı olduğunu bilip ne kadar harcadığına bakabilicez

id_x:'event_id'
id_y:'user_id' duplicate olan bir değer olduğu için silinecek

idleri düzenlemeye geçelim.
PS: Her ne kadar son frame'de olmayacak olsa da düzenli gitmek adına yapılması gereken bir şey.
'''

money_gain_data = money_gain_data.rename(columns={'id_x':'event_id'}).drop('id_y', axis=1)
money_gain_data.head()

,event_id,created_at,user_id,event_name,amount_usd,country
0,6,2024-10-20T00:00:00,2,subscription_started,8.99,NL
1,7,2024-11-19T00:00:00,2,subscription_renewed,8.99,NL
2,8,2024-12-19T00:00:00,2,subscription_renewed,8.99,NL
3,9,2025-01-18T00:00:00,2,subscription_renewed,8.99,NL
4,13,2024-10-21T00:00:00,3,subscription_started,4.99,TR


In [619]:
revenue_by_country = (money_gain_data.groupby('country')['amount_usd'] #Country olarak gruplayıp sadece paraları listeliyoruz.
                     .sum() #Listelerdiğimiz paraları topluyoruz.
                     .reset_index() #Series yerine frame'e çekmek için index resetlioyurz PS: Gerekli olan bir şey değil ama Frame halindeyken gösterim ve okumak daha rahat oluyor.
                     .rename(columns={'amount_usd': 'total_revenue'})  # 'amount_usd' yerine 'total_revenue' kullanmak daha doğru bir yaklaşım.
                    )

#### Task 1 - A:

In [620]:
revenue_by_country #DataFrame halinde cevap.

,country,total_revenue
0,NL,3335.29
1,TR,2275.44
2,US,4035.96


### Task 2
Calculate the total number of trials given to users who came from Instagram.

In [621]:
#Önce instagramdan gelen kullanıcıları bulalım.

instagram_users = users_df[users_df['attribution_source'] == 'instagram']

instagram_users.head()

,id,created_at,attribution_source,country,name
0,1,2024-05-07T00:00:00,instagram,US,Eve Brown
1,2,2024-10-12T00:00:00,instagram,NL,Frank Moore
6,7,2024-10-21T00:00:00,instagram,US,Henry Moore
11,12,2024-01-08T00:00:00,instagram,US,Frank Miller
14,15,2024-07-10T00:00:00,instagram,US,Bob Miller


In [622]:
instagram_trials = events_df[(events_df['event_name'] == 'trial_started') & (events_df['user_id'].isin(instagram_users['id'])) ]

#İnstagram userı olup event'lerde trail başlatan kişileri bulalım.

instagram_trials

,id,created_at,user_id,event_name,amount_usd
1,2,2024-05-12T00:00:00,1,trial_started,NaN
4,5,2024-10-13T00:00:00,2,trial_started,NaN
44,45,2024-01-14T00:00:00,12,trial_started,NaN
63,64,2024-07-12T00:00:00,15,trial_started,NaN
66,67,2024-04-28T00:00:00,16,trial_started,NaN
...,...,...,...,...,...
3432,3433,2025-01-17T00:00:00,989,trial_started,NaN
3438,3439,2024-11-26T00:00:00,990,trial_started,NaN
3448,3449,2025-02-09T00:00:00,992,trial_started,NaN
3458,3459,2024-10-04T00:00:00,995,trial_started,NaN


In [623]:
#Her ne kadar total row sayısını görebiliyor olsak da net bir gösterim olması için kaç row bulunduğuna bakalım.

total_instagram_trials = len(instagram_trials)

#### Task 2 - A:

In [624]:
total_instagram_trials

210

### Task 3
Create a new column named 'acquisition_channel' by categorizing users based on their 'attribution_source':
   - 'Paid': users from instagram or tiktok
   - 'Organic': users from organic sources

In [625]:
# Eğer sadece instagram ve tiktok 'Paid' geri kalan yerler organic ise öncelikle hepsini 'Organic' yapalım
updated_users_df = users_df.copy() #Asıl df'imizi bozmaması adına yeni bir df'e kopyalayalım.
updated_users_df['acquisition_channel'] = 'Organic' # Bütün column'a 'Organic' verelim.

updated_users_df.head()

,id,created_at,attribution_source,country,name,acquisition_channel
0,1,2024-05-07T00:00:00,instagram,US,Eve Brown,Organic
1,2,2024-10-12T00:00:00,instagram,NL,Frank Moore,Organic
2,3,2024-10-15T00:00:00,tiktok,TR,Ivy Anderson,Organic
3,4,2024-08-28T00:00:00,tiktok,TR,Alice Brown,Organic
4,5,2024-04-03T00:00:00,organic,NL,Bob Moore,Organic


In [626]:
#Şimdi tiktok ve instagramı padi yapma vakti.


updated_users_df.loc[updated_users_df['attribution_source'].isin(['instagram', 'tiktok']), 'acquisition_channel'] = 'Paid'


#Eğer attr_src instagram veya tiktok ise(liste olarak verdiğimiz için .isin kullandık) o satırların(en başta .loc kullandık) acq_chnl'larını 'Paid' yapalım


updated_users_df.head()

,id,created_at,attribution_source,country,name,acquisition_channel
0,1,2024-05-07T00:00:00,instagram,US,Eve Brown,Paid
1,2,2024-10-12T00:00:00,instagram,NL,Frank Moore,Paid
2,3,2024-10-15T00:00:00,tiktok,TR,Ivy Anderson,Paid
3,4,2024-08-28T00:00:00,tiktok,TR,Alice Brown,Paid
4,5,2024-04-03T00:00:00,organic,NL,Bob Moore,Organic


#### Task 3 - A:

In [627]:
updated_users_df

,id,created_at,attribution_source,country,name,acquisition_channel
0,1,2024-05-07T00:00:00,instagram,US,Eve Brown,Paid
1,2,2024-10-12T00:00:00,instagram,NL,Frank Moore,Paid
2,3,2024-10-15T00:00:00,tiktok,TR,Ivy Anderson,Paid
3,4,2024-08-28T00:00:00,tiktok,TR,Alice Brown,Paid
4,5,2024-04-03T00:00:00,organic,NL,Bob Moore,Organic
...,...,...,...,...,...,...
997,998,2025-02-01T00:00:00,instagram,TR,Bob Davis,Paid
998,999,2024-12-24T00:00:00,organic,NL,Charlie Davis,Organic
999,1000,2025-02-13T00:00:00,organic,NL,Jack Anderson,Organic
1000,1001,2025-02-16T00:00:00,instagram,US,Bruce Wayne,Paid


### Task 4
Analyze the trial-to-subscription conversion rate:
   - Calculate the overall conversion rate
   - Break down the conversion rate by attribution_source

#### Task 4-1

In [628]:
#Conversion rate'i bulmak için kaç adet trial başlatıldığını ve kaç adet subs başlatıldığını bulalım

total_trials = events_df[events_df['event_name'] == 'trial_started'].shape[0]
total_conversions = events_df[events_df['event_name'] == 'subscription_started'].shape[0]

In [629]:
overall_conversion_rate = str(round((total_conversions / total_trials * 100), 2))+'%'

##### Task 4-1 - A:

In [630]:
overall_conversion_rate

'70.53%'

#### Task 4-2

In [631]:
#Öncelikle attr_src ile eventleri birleştirelim.

events_with_source = events_df.merge(
    users_df[['id', 'attribution_source']], 
    left_on='user_id', 
    right_on='id'
)

events_with_source
#Her şey yerli yerinde gözüküyor

,id_x,created_at,user_id,event_name,amount_usd,id_y,attribution_source
0,1,2024-05-07T00:00:00,1,app_install,NaN,1,instagram
1,2,2024-05-12T00:00:00,1,trial_started,NaN,1,instagram
2,3,2024-05-24T00:00:00,1,trial_cancelled,NaN,1,instagram
3,4,2024-10-12T00:00:00,2,app_install,NaN,2,instagram
4,5,2024-10-13T00:00:00,2,trial_started,NaN,2,instagram
...,...,...,...,...,...,...,...
3481,3482,2025-02-25T00:00:00,1000,trial_cancelled,NaN,1000,organic
3482,3483,2025-02-25T00:00:00,1001,app_install,NaN,1001,instagram
3483,3484,2025-02-25T00:00:00,1001,trial_started,NaN,1001,instagram
3484,3485,2025-02-25T00:00:00,1001,subscription_started,9.99,1001,instagram


In [632]:
# Conversion rate hesaplaması için trail ve subs'ları attr_src bazlı sayılarını alalım
conversion_df = pd.DataFrame({
    'trials': events_with_source[events_with_source['event_name'] == 'trial_started']
                .groupby('attribution_source')
                .size(),
    'subscriptions': events_with_source[events_with_source['event_name'] == 'subscription_started']
                .groupby('attribution_source')
                .size()
})

'''
'trials' isminde column'a trial sayılarını,
'subscriptions' isminde bir column'a subs sayılarını atadık
'''



conversion_df

,trials,subscriptions
attribution_source,,
instagram,210,149
organic,236,169
tiktok,236,163


In [633]:
#Conversion rate'i bulmak için istediğimiz verilere sahibiz, o zaman hesaplayalım
conversion_df['conversion_rate'] = round((conversion_df['subscriptions'] / conversion_df['trials'] * 100), 2)

conversion_df['conversion_rate'] = conversion_df['conversion_rate'].astype(str) + '%' # Yüzdelik bir değer hesapladığımız için sonuna % ekleyelim

##### Task 4-2 - A:

In [634]:
conversion_df

,trials,subscriptions,conversion_rate
attribution_source,,,
instagram,210,149,70.95%
organic,236,169,71.61%
tiktok,236,163,69.07%


#### Task 4-Opsionel

Bu opsionel kısımda attr_src'ların overall'un yüzde kaçı olduğunu gösteren ufak bir hesap yapacağız

In [635]:
#Öncelikle kaç adet sub nereden geliyor onu bulalım, çoktan bulduk ama bu sefer amacımız farklı olduğu için tekrardan yapacağız.
subs_df = pd.DataFrame({
    'subscriptions' :events_with_source[events_with_source['event_name'] == 'subscription_started']
                        .groupby('attribution_source')
                        .size()})

subs_df

,subscriptions
attribution_source,
instagram,149
organic,169
tiktok,163


In [636]:
#Toplam subs sayısını bulalım
total_subs = subs_df.sum()
total_subs

subscriptions    481
dtype: int64

In [637]:
# Şimdi toplam subs sayısını oranlayıp toplam subs'a göre yüzde kaç olduklarını bulalım.
subscription_percentage = round((subs_df / total_subs * 100), 2)
subscription_percentage = subscription_percentage['subscriptions'].astype(str) + '%' #İşlemlerin sonuna %'lik işareti koyalım

subscription_percentage



attribution_source
instagram    30.98%
organic      35.14%
tiktok       33.89%
Name: subscriptions, dtype: object

In [638]:
#Hala bir series halinde tutuyoruz bunu frame haline çevirelim
subscription_percentage_df = pd.DataFrame(subscription_percentage) # Frame haline getirdik

subscription_percentage_df = subscription_percentage_df.rename(columns={'subscriptions':'percentage'}) # Column adını yüzdelik bir değer bulduğumuz için percentage olarak değiştirelim.

##### Task 4-Opsionel - A:

In [639]:
subscription_percentage_df #Total value round to %100.01 because of rounding error but it's not a significant error so lets continue with other task.

,percentage
attribution_source,
instagram,30.98%
organic,35.14%
tiktok,33.89%


### Task 5

Calculate the median subscription duration (in months) for each country

In [640]:
#First of all merge subs realted columns with user_df to make a whole table.
subscription_events = events_df[
    events_df['event_name'].isin(['subscription_started', 'subscription_renewed'])
].merge(
    users_df[['id', 'country']], 
    left_on='user_id', 
    right_on='id'
)

subscription_events

,id_x,created_at,user_id,event_name,amount_usd,id_y,country
0,6,2024-10-20T00:00:00,2,subscription_started,8.99,2,NL
1,7,2024-11-19T00:00:00,2,subscription_renewed,8.99,2,NL
2,8,2024-12-19T00:00:00,2,subscription_renewed,8.99,2,NL
3,9,2025-01-18T00:00:00,2,subscription_renewed,8.99,2,NL
4,13,2024-10-21T00:00:00,3,subscription_started,4.99,3,TR
...,...,...,...,...,...,...,...
1226,3468,2025-04-07T00:00:00,996,subscription_renewed,4.99,996,TR
1227,3476,2025-01-03T00:00:00,999,subscription_started,8.99,999,NL
1228,3477,2025-02-02T00:00:00,999,subscription_renewed,8.99,999,NL
1229,3478,2025-03-04T00:00:00,999,subscription_renewed,8.99,999,NL


In [641]:
# So we have enough info for calculating months per user so lets calculate it
subscription_duration = (subscription_events
    .groupby(['user_id', 'country'])
    .size()  # events per user
    .reset_index(name='months') # Lets change that columns name from 0 to months for better understanding
)

subscription_duration

,user_id,country,months
0,2,NL,4
1,3,TR,4
2,8,TR,2
3,9,US,2
4,10,US,2
...,...,...,...
476,992,US,5
477,995,US,3
478,996,TR,3
479,999,NL,3


In [642]:
# Now we can group every country to get median of our duration for each country
median_duration = subscription_duration.groupby('country')['months'].median()


median_duration_df = pd.DataFrame(median_duration) #Lets change our answer from series to frame

#### Task 5 - A:

In [643]:
median_duration_df

,months
country,
NL,2.0
TR,3.0
US,2.0


### Task 6
Calculate the Average Lifetime Value (LTV) by country

In [644]:
#Lets merge subs related columns with user_df to get big picture

subscription_revenue = events_df[
    events_df['event_name'].isin(['subscription_started', 'subscription_renewed'])
].merge(
    users_df[['id', 'country']], 
    left_on='user_id', 
    right_on='id'
)

subscription_revenue.head()

,id_x,created_at,user_id,event_name,amount_usd,id_y,country
0,6,2024-10-20T00:00:00,2,subscription_started,8.99,2,NL
1,7,2024-11-19T00:00:00,2,subscription_renewed,8.99,2,NL
2,8,2024-12-19T00:00:00,2,subscription_renewed,8.99,2,NL
3,9,2025-01-18T00:00:00,2,subscription_renewed,8.99,2,NL
4,13,2024-10-21T00:00:00,3,subscription_started,4.99,3,TR


In [645]:
# Calculating every users ltv

user_ltv = (subscription_revenue
    .groupby(['user_id', 'country'])['amount_usd']
    .sum()
    .reset_index()
)

user_ltv.head()

,user_id,country,amount_usd
0,2,NL,35.96
1,3,TR,19.96
2,8,TR,9.98
3,9,US,19.98
4,10,US,19.98


In [646]:
# Now we have everything we need so calculate country based average revenue

avg_ltv_by_country = pd.DataFrame(user_ltv
    .groupby('country')['amount_usd']
    .mean()
    .round(2)
)

avg_ltv_by_country['amount_usd'] = '$'+avg_ltv_by_country['amount_usd'].astype(str)
avg_ltv_by_country = avg_ltv_by_country.rename(columns={'amount_usd':'average_revenue'})


#### Task 6 - A:

In [647]:
avg_ltv_by_country


,average_revenue
country,
NL,$22.38
TR,$13.31
US,$25.07


### Task 7
Predict the churn probability for user #1002 (Clark Kent)

Use any relevant features from the dataset
Explain your model selection and feature engineering process
Provide confidence intervals if applicable

In [698]:
#Importing libraries

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import numpy as np

In [699]:
# Making amount_usd nulls to 0 for preventing error while modelling

events_df['amount_usd'] = events_df['amount_usd'].fillna(0)

In [700]:
# Creting new feature and onehot encoding categorical values

def create_features(events_df, users_df):
    user_features = []
    
    for user_id in users_df['id']:
        user_events = events_df[events_df['user_id'] == user_id] #User's events table features
        user_info = users_df[users_df['id'] == user_id].iloc[0] #User's users table features
        
        features = {
            'user_id': user_id, #User_id for that user
            'subscription_count': user_events[user_events['event_name'].isin(['subscription_started', 'subscription_renewed'])].shape[0], #How many times user subs
            'total_revenue': user_events['amount_usd'].sum(), # How much payment done by user
            'had_trial': any(user_events['event_name'] == 'trial_started'), # Is user had any trial
            'trial_converted': any(user_events['event_name'] == 'trial_started') and  
                             any(user_events['event_name'] == 'subscription_started'), #Checking if user had trial started and sub started 
            'is_organic': user_info['attribution_source'] == 'organic', # Is user attr_src organic
            'is_instagram': user_info['attribution_source'] == 'instagram' ,# Is user attr_src instagram
            'is_tiktok': user_info['attribution_source'] == 'tiktok', # Is user attr_src tiktok
            'country_US': user_info['country'] == 'US', # Is user country US
            'country_TR': user_info['country'] == 'TR', # Is user country TR
            'country_NL': user_info['country'] == 'NL', # Is user country NL
            'churned': any(user_events['event_name'] == 'subscription_cancelled') # If user has a subs_cancel that user is churned
        }
        user_features.append(features)
    
    return pd.DataFrame(user_features)

In [701]:
features_df = create_features(events_df, users_df) # Making df with our function above

In [702]:
features_df

,user_id,subscription_count,total_revenue,had_trial,trial_converted,is_organic,is_instagram,is_tiktok,country_US,country_TR,country_NL,churned
0,1,0,0.00,True,False,False,True,False,True,False,False,False
1,2,4,35.96,True,True,False,True,False,False,False,True,True
2,3,4,19.96,True,True,False,False,True,False,True,False,True
3,4,0,0.00,True,False,False,False,True,False,True,False,False
4,5,0,0.00,False,False,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
997,998,0,0.00,False,False,False,True,False,False,True,False,False
998,999,3,26.97,True,True,True,False,False,False,False,True,True
999,1000,0,0.00,True,False,True,False,False,False,False,True,False
1000,1001,1,9.99,True,True,False,True,False,True,False,False,False


In [703]:
# Our prediction value is 'churned' so we split df for training 
X = features_df.drop(['user_id', 'churned'], axis=1) # User_id is irrelevant and our target is churned we drop those values
y = features_df['churned'] # Our target value

In [704]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #Standart train test spliting

In [705]:
# We scale our features make them scaled because some features like total_rev will have higher impact on our target becasue most of our features is 0,1(False,True)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [706]:
'''
I choosed LogisticRegression because our churned value is True or False a binary decision and 
I want to take a look at the feature importance for business strategies to determine in real life approach
'''

model = LogisticRegression(random_state=42) 
model.fit(X_train_scaled, y_train)



LogisticRegression(random_state=42)

In [707]:
user_1002_features = features_df[features_df['user_id'] == 1002].drop(['user_id', 'churned'], axis=1) # Get features for user_id 1002
user_1002_scaled = scaler.transform(user_1002_features) # Scale values
churn_probability = model.predict_proba(user_1002_scaled)[0][1] # Get churn probability for user 1002

In [708]:
# We have already found the probabilty but lest analayze more
feature_impact = pd.DataFrame({
    'feature': X.columns, #Features in df
    'impact': model.coef_[0] # Coefficient for its impact value it can be positive or negative
})

feature_impact = feature_impact.sort_values('impact', ascending=False) # Sort impacting values by their values

In [709]:
# Lets make its importance to churn its almost a same df but with absolute value on impact -> importance column

feature_importance = pd.DataFrame({
    'feature': feature_impact['feature'],
    'importance': feature_impact['impact'].abs()
})

feature_importance = feature_importance.sort_values('importance', ascending=False)

#### Task 7 - A:


In [710]:
churn_probability

0.0005141024656188886

In [711]:
'''
Positive values are have positive impact to churn so if users subs_count is increasing they are more likly to churn, in real life it may not be like that but with this amount of
data we found this correlation. But there is a meaningful insights too, users from TR is more likely to chrun than other countries.
'''

feature_impact

,feature,impact
0,subscription_count,4.540945
1,total_revenue,3.838298
8,country_TR,0.638214
3,trial_converted,0.230802
2,had_trial,0.134874
5,is_instagram,0.064670
6,is_tiktok,-0.020954
4,is_organic,-0.041856
9,country_NL,-0.251924
7,country_US,-0.404605


In [712]:
'''
Importance of values, basicly same df with absoulte values to determine which feature has higher impact,
original value may be negative and can affect more than other positive effects, so it may make more sense to make negatives less than making positives more attractive.
'''
feature_importance 

,feature,importance
0,subscription_count,4.540945
1,total_revenue,3.838298
8,country_TR,0.638214
7,country_US,0.404605
9,country_NL,0.251924
3,trial_converted,0.230802
2,had_trial,0.134874
5,is_instagram,0.064670
4,is_organic,0.041856
6,is_tiktok,0.020954


### Task 8
Predict the potential Lifetime Value (pLTV) for user #1001 (Bruce Wayne)

Explain your methodology and assumptions

In [726]:
#Creating a function for prediction

def predict_plv(user_id, churn_model, scaler): #Users_id, our task7 churn proba model, standart scaler

    user_events = events_df[events_df['user_id'] == user_id].sort_values('created_at') # events that user done by timestamp order
    
    
    monthly_payment = user_events[
        user_events['event_name'].isin(['subscription_started', 'subscription_renewed'])
    ]['amount_usd'].iloc[-1] if len(user_events) > 0 else 0 # Last monthly payment
    
    user_features = create_features(events_df, users_df) # function we created at task7
    user_features = user_features[user_features['user_id'] == user_id].drop(['user_id', 'churned'], axis=1) # Dropping columns for prediction
    
    user_features_scaled = scaler.transform(user_features) #Scaling for one that we wanted to test
    churn_probability = churn_model.predict_proba(user_features_scaled)[0][1] #getting probability of churn
    
    
    pltv = (monthly_payment / churn_probability).round(2) # pLTV = monthly payment / churn_probability 
    
    return {
        'user_id': user_id,
        'monthly_payment': monthly_payment,
        'churn_probability': churn_probability,
        'pltv': pltv
    }

In [727]:
prediction = predict_plv(1001, model, scaler) #getting prediction

In [714]:
# Confidence interval calc with bootstrap because its not dependant on distirbution
n_iterations = 10 # How many different sample will generate normally it will be like 1000, but for the sake of our time I make it 10, making it 1000 will make it more stable
pltv_predictions = []

for _ in range(n_iterations):
    indices = np.random.randint(0, len(X_train), len(X_train)) # Random selection for training
    sample_X = X_train_scaled[indices] 
    sample_y = y_train.iloc[indices]
    
    sample_model = LogisticRegression(random_state=42) # We use logistic regression to get churn prob for this dataset
    sample_model.fit(sample_X, sample_y) #Model fitting
    
    user_features = create_features(events_df, users_df) #Function we used in task7 for churn prob
    user_features = user_features[user_features['user_id'] == 1001].drop(['user_id', 'churned'], axis=1) #getting churn prob of 1001 id for this df
    user_features_scaled = scaler.transform(user_features)
    
    churn_prob = sample_model.predict_proba(user_features_scaled)[0][1] # Churn prob
    pltv = (prediction['monthly_payment'] / churn_prob).round(2) # monthly paymen / churn prob
    pltv_predictions.append(pltv) #storing them in list

confidence_interval = np.percentile(pltv_predictions, [2.5, 97.5]) # Finding minimum and maximum potantial pltv

#### Task 8 - A:

In [728]:
print(f"User ID: {prediction['user_id']}")
print(f"Current Monthly Payment: ${prediction['monthly_payment']}")
print(f"Churn Probability: {prediction['churn_probability']:.2%}")
print(f"Lifetime Value: ${prediction['pltv']}")
print(f"pLTV Range: ${confidence_interval[0]:.2f} - ${confidence_interval[1]:.2f}")

User ID: 1001
Current Monthly Payment: $9.99
Churn Probability: 6.23%
Lifetime Value: $160.39
pLTV Range: $121.94 - $208.07
